In [ ]:
!pip install -q langchain-ollama langchain-core requests

In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [ ]:
print(multiply.invoke({'a':1,'b':4}))

In [ ]:
multiply.name

In [ ]:
multiply.description

In [ ]:
multiply.args

In [ ]:
llm = ChatOllama(model='llama3.2:1b')

In [ ]:
llm.invoke('hi')

In [ ]:
llmwithtool = llm.bind_tools([multiply])

In [ ]:
llmwithtool.invoke('hi how are you?')

In [ ]:
query = HumanMessage('can you multiply 3 with 1000')

In [ ]:
message = [query] 

In [ ]:
message

In [ ]:
result = llmwithtool.invoke(message)

In [ ]:
message.append(result)

In [ ]:
message

In [ ]:
toolresult = multiply.invoke(result.tool_calls[0])

In [ ]:
toolresult

In [ ]:
message.append(toolresult)

In [ ]:
message

In [ ]:
llmwithtool.invoke(message).content

In [ ]:
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f' https://v6.exchangerate-api.com/v6/94ecahekswdmkuwxsd9efc5b5cfbc78a481died/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool

def covertcurrency(base_currency_value: int , conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

    return base_currency_value * conversion_rate

In [ ]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'PKR'})

In [ ]:
covertcurrency.invoke({'base_currency_value':11,'conversion_rate':280.09})

In [ ]:
llm = ChatOllama(model='llama3.2:1b')

In [ ]:
llmwithtool = llm.bind_tools([get_conversion_factor,covertcurrency])

In [ ]:
message = [HumanMessage('What is the conversion factor between USD and PKR, and based on that can you convert 10 usd to pkr')]

In [ ]:
message

In [ ]:
aimessage = llmwithtool.invoke(message)

In [ ]:
message.append(aimessage)

In [ ]:
aimessage.tool_calls

In [ ]:
import json

for tool_call in aimessage.tool_calls:
  if tool_call ['name'] == 'get_conversion_factor':
   toolmessage1 = get_conversion_factor.invoke(tool_call)
   conversion_rate = json.loads(toolmessage1.content)['conversion_rate']
   message.append(conversion_rate)

   if tool_call ['name'] == 'covertcurrency':
     tool_call['args']['conversion_rate'] == conversion_rate
     toolmessage2 = covertcurrency.invoke(tool_call)
     message.append(toolmessage2)

In [ ]:
message

In [ ]:
llmwithtool.invoke(message).content